In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import chain
from sklearn.metrics import pairwise_distances
#from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import sent_tokenize
import fasttext as ft
from collections import defaultdict
#from greed import greed_sum_query

from tqdm.notebook import tqdm
import os

import pickle
def save_obj(obj, name):
    pickle.dump(obj,open(name + '.pkl', 'wb'), protocol=4)
    
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

from nltk.corpus import stopwords
sw = stopwords.words('russian')

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.tokenize import word_tokenize

In [2]:
import warnings
warnings.filterwarnings("ignore")
import backoff
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import openai
import re
from langchain_community.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from razdel import tokenize, sentenize
import string
import fasttext as ft


In [10]:
embedding = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-large')

In [11]:
vector_db = FAISS.load_local('final_db', embeddings = embedding, allow_dangerous_deserialization=True )

In [12]:
res = vector_db.similarity_search('Мародерство', k =10)
res

[Document(page_content='Статья 456. Добровольная сдача в плен:Добровольная сдача в плен по трусости или малодушию –   наказывается лишением свободы на срок от пяти до десяти лет.\nСтатья 457. Мародерство:Похищение на поле сражения вещей, находящихся при убитых и раненых (мародерство), –    наказывается лишением свободы на срок от пяти до десяти лет.', metadata={'source': 'C:\\Users\\aliha\\Desktop\\project\\Datas\\Уголовный Кодекс РК.txt'}),
 Document(page_content='Казахстан, конституционными и иными законами Республики Казахстан для непосредственного исполнения функций государства и полномочий государственных органов, в том числе депутат Парламента Республики Казахстан, судья Конституционного Суда Республики Казахстан, судья, Уполномоченный по правам человека в Республике Казахстан, а равно лицо, занимающее согласно законодательству Республики Казахстан о государственной службе политическую государственную должность либо административную государственную должность корпуса "А";17) хищен

In [17]:
def store_to_df(store):
    v_dict=store.docstore._dict
    data_rows=[]
    for k in v_dict.keys():
        doc_name=v_dict[k].metadata['source'].split('\\')[-1]
        content=v_dict[k].page_content
        data_rows.append({"chunk_id":k,"document":doc_name,"content":content})
    vector_df=pd.DataFrame(data_rows)
    return vector_df
        
def delete_document(store,document):
    vector_df=store_to_df(store)
    chunks_list=vector_df.loc[vector_df['document']==document]['chunk_id'].tolist()
    store.delete(chunks_list)

In [18]:
store_to_df(vector_db)

,chunk_id,document,content
0,4e55e1b5-2b80-4569-86d4-b481e2552fdc,Административный процедурно-процессуальный код...,Статья 1. Законодательство Республики Казахста...
1,0d042bc2-1b3a-4f8f-b032-0867e36c7c10,Административный процедурно-процессуальный код...,3. Порядок административного судопроизводства ...
2,acb59fbe-115a-4eb6-b507-c9a5f802b23d,Административный процедурно-процессуальный код...,4. Международные договорные и иные обязательст...
3,8f7f3872-c424-4015-93c1-da178786f0a4,Административный процедурно-процессуальный код...,1. Законодательство Республики Казахстан об ад...
4,1d9da320-62f7-4d85-b156-08c94221db4f,Административный процедурно-процессуальный код...,основанными на Конституции Республики Казахста...
...,...,...,...
81861,cd2bb0ad-7a7b-4564-9e80-d90b9c045526,Экологический кодекс РК.txt,"ст. 161; № 24, ст. 196; 2012 г., № 1, ст. 5; №..."
81862,1bbbe043-cc91-42a6-80b9-b00215e41c29,Экологический кодекс РК.txt,в некоторые законодательные акты Республики Ка...
81863,43f0591d-61d9-436c-b6e8-17cf689dab1c,Экологический кодекс РК.txt,с 1 января 2019 года – для автозаправочных ста...
81864,1c446ac6-9495-447d-8158-be44298ea05e,Экологический кодекс РК.txt,№ 300-VІ (вводится в действие по истечении дес...


In [4]:
model = ft.load_model("cc.ru.300.bin")

In [36]:

class Assistant:
    
    def __init__(self, path, embedding, sw):
        self.db = FAISS.load_local(path, embeddings = embedding, allow_dangerous_deserialization=True )
        self.sw = sw
        self.query = query

    async def greed_sum_query(self, text, query_vec, num_sent=10, min_df=1, max_df=3):
        # Let's take 10% of the most meaningful sentences
        #num_sent = 10 # int(len(text)*0.05) 
        #print('Number of 5% sentences', num_sent)
        #fit a TFIDF vectorizer
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, stop_words=self.sw)
        X1 = vectorizer.fit_transform(text).toarray()

        # query specific TFIDF
        voc = word_tokenize(self.query.lower())
        vectorizer2 = TfidfVectorizer(vocabulary=voc)
        X2 = vectorizer2.fit_transform(text).toarray()

        X = np.concatenate((X1, X2), axis=1)

        text_vecs = np.array([model.get_sentence_vector(s.replace('\n','')) for s in text])

        
        similarity = 1 - pairwise_distances(query_vec.reshape(1, -1), text_vecs, metric='cosine')
        X = X * similarity[0][:, None]

        idx = []
        while sum(sum(X)) != 0:
            ind = np.argmax(X.sum(axis=1))
            idx.append(ind)
            #stop if we have more than 20% of the sentences from the text
            if len(idx) > int(len(text)*0.2):#num_sent:
                break

            #update the matrix deleting the columns corresponding to the words found in previous step
            cols = X[ind]
            col_idx = [i for i in range(len(cols)) if cols[i] > 0]
            X = np.delete(X, col_idx, 1)

        #make a condition to extract a number of sentences or all salient sentences
        if num_sent != 0:
            idx = idx[:num_sent]
        idx.sort()
        summary = [text[i] for i in idx]
        summary_vecs = [text_vecs[i] for i in idx]
        return summary, summary_vecs

    def clean_text(self, str):
        str = re.sub('[\xa0\n]', ' ', str)
        str = str.replace('№', 'номер')
        return str

    def clean_query(self, str):
        clean_query = ""
        for i in self.clean_text(str.lower()).split(" "):
            if i not in self.sw and i not in string.punctuation:
                clean_query += i + " " 
                
        clean_query = clean_query.strip()

        return clean_query

    # def get_section_refs(self, sources, texts):
    #     statyas = []
    #     for name, text in zip(sources, texts):
    #         search_term = text.encode('utf-8')  # Note: binary string for mmap
    #         with open(fr'.\Data\{name}.txt','r',encoding='utf-8') as file:
    #             content = file.read()
    #             index = content.find(text)
    #             pattern = r'Статья \d+'
    #             match = re.search(pattern, content[index:index+12])
    #             index += 12
    #             for s in range(index, 10, -1):
    #                 temp = content[s-11:s] 
    #                 match = re.search(pattern, temp)
    #                 if match:
    #                     statya = match.group()
    #                     statyas.append(statya)
    #                     break
    #     return statyas

    def get_refs(self, final_summary_texts, t_summarized_each):
        refs = set()
        for s in final_summary_texts:
            for i, d in enumerate(t_summarized_each) :
                if s in self.clean_text(d):
                    refs.add(i)
        return refs

    def paragraphs_split(self, final_sum):
        sum_sent_vecs = [np.array(model.get_sentence_vector(s.replace('\n',''))) for s in final_sum] 
        sims = []
        for i in range(1, len(sum_sent_vecs)):
            sims.append(1 - pairwise_distances(sum_sent_vecs[i].reshape(1, -1), sum_sent_vecs[i-1].reshape(1, -1), metric='cosine')[0][0])
        treshld = 0.8
        par_begin = np.array(sims) <= treshld
        par_begin_idx = np.array(range(len(sims)))[par_begin]

        return par_begin_idx
    
    async def get_answer(self, q):
        #GREED SUMMARY
        query = self.clean_query(q)
        query_vec = model.get_sentence_vector(query) 
        
        docs = self.db.similarity_search(query=query, k = 10)
        t = [ i.page_content for i in docs ]
        sources = [ i.metadata['source'].split('\\')[-1].split('.')[0] for i in docs ]
        #Summarizing each section
        t_summarized_each, sentences_t_summarized_together, sentences_t_summarized_together_vecs = ([] for i in range(3))
        for section in t:
            section_summmary, section_summmary_vec = await self.greed_sum_query([i.text for i in list(sentenize(self.clean_text(section))) ], query_vec, num_sent=10)
            t_summarized_each.append(" ".join(section_summmary))
            sentences_t_summarized_together += section_summmary
            sentences_t_summarized_together_vecs += section_summmary_vec

        text_vecs = np.array(sentences_t_summarized_together_vecs)
        
        similarity = 1 - pairwise_distances(query_vec.reshape(1, -1), text_vecs, metric='cosine')
#         print(similarity)
        indexes = [i[1] for i in sorted([(j, i) for i, j in enumerate(similarity[0])], reverse=True) if i[0] >= 0.3]
        final_summary_texts = [sentences_t_summarized_together[i] for i in indexes]
        # statyas = self.get_section_refs(sources, t)
        
        # 4. Get references for each sentence in the resulting summary
        refs = self.get_refs(final_summary_texts, t_summarized_each)
                    
        final_sum = [s + '[%s]'%(r+1) for s, r in zip(final_summary_texts, refs)]
        ref_list = ['%s. '%(r+1)+sources[r] for r in refs]
        # 5. Separate the text into paragraphs
        ## Find paragraph boundaries
    
        ### consecutive sentence pair similarity
        par_begin_idx = self.paragraphs_split(final_sum)

        pars = self.partition(final_sum, list(par_begin_idx))
        par_tits = [self.get_title(p, query) for p in pars]
        return par_tits, pars, ref_list
    
    
    ## Make titles for paragraphs
    def get_title(self, par, query):
        
        vectorizer = TfidfVectorizer(ngram_range=(3, 3),stop_words=sw)
        X = vectorizer.fit_transform(par).toarray().sum(axis=0)

        titles = vectorizer.get_feature_names_out()

        #query_vec =  np.array(embedding.embed_query(query)) #
        query_vec = model.get_sentence_vector(query) 
        #text_vecs =  np.array([embedding.embed_query(s) for s in titles]) #
        text_vecs = np.array([model.get_sentence_vector(s.replace('\n','')) for s in titles])

        similarity = 1 - pairwise_distances(query_vec.reshape(1, -1), text_vecs, metric='cosine')

        return titles[np.argmax(similarity)].upper() 
    
    def partition(self, alist, indices):
        return [alist[i:j] for i, j in zip([0]+indices, indices+[None])]




In [37]:
db_path = "final_db"
query = "Мародерство"
ass = Assistant(db_path, embedding, sw)

In [38]:
# 6. Put together text with inline references and a reference list
par_titles, pars, ref_list = await ass.get_answer(query)

answer_str = f'Ответ по запросу: {query}\n'

for t,p in zip(par_titles, pars):
    answer_str += t + '\n' + '. '.join(p) + '\n\n'

answer_str += 'СПИСОК ИСПОЛЬЗОВАННОЙ ЛИТЕРАТУРЫ:\n'

for r in ref_list:
    answer_str += r + '\n\n'

In [39]:
print(answer_str)

Ответ по запросу: Мародерство
УМЫШЛЕННОЕ УНИЧТОЖЕНИЕ ПОВРЕЖДЕНИЕ
нарушения норм международного гуманитарного права во время вооруженных конфликтов (статья 165); бездействие либо дача преступного приказа во время вооруженного конфликта (статья 166); геноцид (статья 168); экоцид (статья 169); участие в иностранных вооруженных конфликтах (статья 172); государственная измена (статья 175); шпионаж (статья 176); грабеж (частьчетвертая статьи 191); разбой (частивторая,третья ичетвертая статьи 192); хищение предметов, имеющих особую ценность (частивторая итретья статьи 193); вымогательство (частитретья ичетвертая статьи 194); неправомерное завладение автомобилем или иным транспортным средством без цели хищения (частитретья ичетвертая статьи 200); умышленное уничтожение или повреждение чужого имущества (частьтретья статьи 202); умышленное уничтожение или[2]. к отказу от ее совершения (части вторая и третья статьи 248); рейдерство (часть третья статьи 249); захват заложника (статья 261); создани

In [32]:
template = """Вопрос: Штраф за проезд красного света светофора?

В соответствии 599 статьей Об административных правонарушениях Кодекса РК
Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика, за исключением случаев, предусмотренных частью первой статьи 607 настоящего Кодекса, –   влечет штраф в размере десяти месячных расчетных показателей.

СПИСОК ИСПОЛЬЗОВАННОЙ ЛИТЕРАТУРЫ:
1. Кодекс РК Об административных правонарушениях
Статья 595
"""

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
# gpt-3.5-turbo-0125
# gpt-4-0125-preview
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125",  api_key = api_key)
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты система gpt 3.5, разработанная OpenAI. Так как ты мощная система гпт, ты должен отвечать на должном уровне. "),
        ("assistant", "Ты ассистент, который должен сделать то, что просит пользователь. Не делай лишних движений.Ты должен вернуть в таком виде, пример:{template} "),
#         ("user", "Тебе дается уже готовый текст по юриспруденции Республики Казахстан. Сделай следущее: Убери предложения, которые не отвечают на этот вопрос {query},но оставляй синонимы. Затем напиши складный текст на более менее простом языке и покажи к нему релевантные источники\n {user_input}")
        ("user", "Тебе дается текст по юриспруденции Республики Казахстан. Убери предложения, которые не отвечают на вопрос юриспруденции {query}, оставив синонимы. Затем напиши официальный текст, который будет понятен пользователю и полностью отвечать на вопросы о юриспруденции, содержащиеся в {user_input}. Укажи релевантные источники.")
    ]
)
human_template=answer_str
chain =  chat_prompt | llm
answer = chain.invoke({
    "query":f"{query}", "user_input":f"{human_template}", "template":f"{template}"})

print(answer.content)


УБИЙСТВО ПРОТИВОПРАВНОЕ УМЫШЛЕННОЕ

Убийство, что означает незаконное умышленное причинение смерти другому человеку, наказывается лишением свободы на срок от восьми до пятнадцати лет. Убийство с особой жестокостью, садизмом, издевательством или мучениями для потерпевшего также подпадает под уголовную ответственность.

УБИЙСТВО СТАТЬЯ 96

Убийство, совершенное в определенных обстоятельствах, включая убийство в состоянии аффекта или причинение смерти по неосторожности, также является уголовно наказуемым деянием.

УГРОЗА НАСИЛЬСТВЕННЫЕ ДЕЙСТВИЯ

Действия, направленные на угрозу жизни других людей, особенно если они способны причинить серьезный вред общественной безопасности или вызвать международные проблемы, также подпадают под уголовную ответственность.

Источники:
1. Уголовный Кодекс Республики Казахстан


In [42]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings


In [105]:
dir_list = os.listdir(r'C:\Users\aliha\Desktop\project\Datas')

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
document  = TextLoader(f'C:\Users\aliha\Desktop\NLP\parser\docs\2011 - 2013 жылдарға арналған республикалық бюджет туралы.docx', encoding='utf-8').load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
)

text = text_splitter.split_documents(document)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3472919169.py, line 3)

In [23]:
from langchain_community.document_loaders import Docx2txtLoader

In [33]:
loader = Docx2txtLoader(r'C:\Users\aliha\Desktop\NLP\parser\docs\adilet_organdary_turaly.doc')

In [34]:
data = loader.load()

BadZipFile: File is not a zip file

In [ ]:
data

In [106]:
docs = []
for path in dir_list:
    document  = TextLoader(fr'C:\Users\aliha\Desktop\project\Datas\{path}', encoding='utf-8').load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=131,
    )
    
    text = text_splitter.split_documents(document)
    
    docs.extend(text)
    


In [4]:
# vector_db = FAISS.from_documents(documents=docs, embedding=embedding)
# vector_db.save_local("./final_db")

NameError: name 'docs' is not defined

In [5]:
vector_db = FAISS.load_local('final_db', embeddings = embedding, allow_dangerous_deserialization=True )
vector_db.similarity_search('срок за похищение', k=10)

[Document(page_content='либо в материальной или иной зависимости от виновного, а равно лица, похищенного либо захваченного в качестве заложника;   2) в отношении двух или более лиц;   3) в отношении женщины, заведомо для виновного находящейся в состоянии беременности;   4) исключен Законом РК от 17.03.2023 № 212-VII (вводится в действие по истечении шестидесяти календарных дней после дня его первого официального опубликования).  5) по найму;   6) по мотиву социальной, национальной, расовой, религиозной ненависти или вражды, –    наказывается ограничением свободы на срок от четырех до семи лет либо лишением свободы на тот же срок.   Сноска. Статья 110 с изменениями, внесенными законами РК от 12.07.2018 № 180-VІ (вводится в действие по истечении десяти календарных дней после дня его первого официального', metadata={'source': 'C:\\Users\\aliha\\Desktop\\project\\Datas\\Уголовный Кодекс РК.txt'}),
 Document(page_content='удостоверяющих личность потерпевшего, –    наказывается лишением своб

In [163]:
# vector_db = FAISS.from_documents(documents=docs, embedding=embedding)
# vector_db.save_local("./current_laws")

In [63]:
# vector_db.similarity_search('убийство', k=10)

NameError: name 'vector_db' is not defined

In [47]:
query = "Будут ли штрафные санкции за несвоевременную уплату налога?"

In [48]:
vector_db = FAISS.load_local('final_db', embeddings = embedding, allow_dangerous_deserialization=True)
res = vector_db.similarity_search(query=query, k = 100)

In [49]:
df = pd.DataFrame()
df['text'] = [p.page_content for p in res]

In [50]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] =df['text'].apply(get_embedding)



In [51]:
df.head()

,text,ada_embedding
0,от начисленной суммы налогов и других обязател...,"[0.012270797044038773, -0.013004198670387268, ..."
1,Статья 205. Несвоевременная или неполная уплат...,"[-0.01137534249573946, 0.03087187372148037, 0...."
2,– влечет штраф на физических лиц в размере ...,"[0.00012873481318820268, -0.016183365136384964..."
3,или несвоевременно перечисленного налога и дру...,"[0.022077539935708046, 0.017081642523407936, 0..."
4,или его должностных лиц в рамках рассмотрения ...,"[-0.0007654961082153022, -0.011162828654050827..."


In [52]:
df.to_csv("embeddings.csv")

In [53]:
def distances_from_embeddings(embedding1, embeddings2, distance_metric="cosine"):
    """
    Function to compute distances between two sets of embeddings.
    """
    distances = []
    for embedding2 in embeddings2:
        # Compute cosine distance between embeddings
        distance = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
        distances.append(distance)
    return distances

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    # Get embeddings for the question text
    question_embedding = get_embedding(question)

    # Compute cosine distances between question embedding and embeddings in the DataFrame
    distances = distances_from_embeddings(question_embedding, df["ada_embedding"].values)

    # Make a copy of the DataFrame and add a "distances" column
    df_copy = df.copy()
    df_copy["distances"] = distances

    # Sort the copied DataFrame by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=False, inplace=True)
    return df_copy

In [54]:
get_rows_sorted_by_relevance(query, df)

,text,ada_embedding,distances
0,от начисленной суммы налогов и других обязател...,"[0.012270797044038773, -0.013004198670387268, ...",0.590462
1,Статья 205. Несвоевременная или неполная уплат...,"[-0.01137534249573946, 0.03087187372148037, 0....",0.586023
3,или несвоевременно перечисленного налога и дру...,"[0.022077539935708046, 0.017081642523407936, 0...",0.577612
2,– влечет штраф на физических лиц в размере ...,"[0.00012873481318820268, -0.016183365136384964...",0.576873
4,или его должностных лиц в рамках рассмотрения ...,"[-0.0007654961082153022, -0.011162828654050827...",0.570724
...,...,...,...
97,от уплаты налогов;Примечание ИЗПИ!В подпункт 9...,"[0.008501759730279446, 0.005261555314064026, -...",0.507702
96,налогового обязательства по уплате налогов и (...,"[0.012801192700862885, -0.004330854397267103, ...",0.507696
94,отражения их результатов в бухгалтерском учете...,"[0.013536610640585423, -0.007772299461066723, ...",0.507622
93,"дней со дня, следующего за днем вручения уведо...","[0.01951918564736843, 0.008319652639329433, 0....",0.507579


In [55]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context:

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


In [56]:
print(create_prompt(query, df, 200))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context:



---

Question: Будут ли штрафные санкции за несвоевременную уплату налога?
Answer:


In [57]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=500):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """

    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = client.completions.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response.choices[0].text
    except Exception as e:
        print(e)
        return ""


In [58]:
custom_ukraine_answer = answer_question(query, df)
print(custom_ukraine_answer)

 Да


In [59]:
res

[Document(page_content='от начисленной суммы налогов и других обязательных платежей в бюджет.   2. Занижение налогоплательщиком сумм текущих платежей в расчете, если это действие не содержит признаков уголовно наказуемого деяния, –    влечет штраф на субъектов малого предпринимательства или некоммерческие организации, на субъектов среднего предпринимательства в размере тридцати, на субъектов крупного предпринимательства – в размере пятидесяти процентов от заниженной суммы текущих платежей.   3. Превышение суммы фактически исчисленного корпоративного подоходного налога за налоговый период над суммой исчисленных авансовых платежей в течение налогового периода в размере более двадцати процентов, если это действие не содержит признаков уголовно наказуемого деяния, –    влечет штраф в размере двадцати процентов от', metadata={'source': 'C:\\Users\\aliha\\Desktop\\project\\Datas\\Кодекс РК Об административных правонарушениях.txt'}),
 Document(page_content='Статья 205. Несвоевременная или неп

In [60]:
df

,text,ada_embedding
0,от начисленной суммы налогов и других обязател...,"[0.012270797044038773, -0.013004198670387268, ..."
1,Статья 205. Несвоевременная или неполная уплат...,"[-0.01137534249573946, 0.03087187372148037, 0...."
2,– влечет штраф на физических лиц в размере ...,"[0.00012873481318820268, -0.016183365136384964..."
3,или несвоевременно перечисленного налога и дру...,"[0.022077539935708046, 0.017081642523407936, 0..."
4,или его должностных лиц в рамках рассмотрения ...,"[-0.0007654961082153022, -0.011162828654050827..."
...,...,...
95,"административного взыскания, – влекут штраф ...","[-0.017096884548664093, -0.027679461985826492,..."
96,налогового обязательства по уплате налогов и (...,"[0.012801192700862885, -0.004330854397267103, ..."
97,от уплаты налогов;Примечание ИЗПИ!В подпункт 9...,"[0.008501759730279446, 0.005261555314064026, -..."
98,Статья 123. Общие положения:1. Погашение задол...,"[-0.002414420247077942, -0.012278788723051548,..."


In [43]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [76]:
import os
import unicodedata

In [90]:
dir_list = os.listdir('C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt')

In [91]:
dir_list = [txt for txt in dir_list if txt.endswith('txt')]

In [97]:
for path in dir_list:
    with open(f'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt\\{path}', 'r', encoding='utf-8') as f:
        text = f.read()
#         line = ' '.join(line.split())

    new_str = unicodedata.normalize("NFKD", text)
    new_str = new_str.replace('\n\n', '\n')
    res = re.sub(r'\n\s+', '\n', new_str)
    
# print(new_str)
    with open(f'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt_format\\{path}', 'w', encoding='utf-8') as f:
        f.write(res)


In [98]:
docs = []
for path in dir_list:
    document  = TextLoader(f'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt_format\\{path}', encoding='utf-8').load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150
    )

    text = text_splitter.split_documents(document)
    
    docs.extend(text)

In [102]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
document  = TextLoader(f'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt_format\\Қазақстан Республикасының Қылмыстық кодексi.txt', encoding='utf-8').load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
)

text = text_splitter.split_documents(document)

In [111]:
text

[Document(page_content='Қазақстан Республикасының Қылмыстық кодексi\n1-бап. Қазақстан Республикасының қылмыстық заңнамасы\n1. Қазақстан Республикасының қылмыстық заңнамасы Қазақстан Республикасының осы Қылмыстық кодексiнен тұрады. Қылмыстық жауаптылықты көздейтiн өзге заңдар осы Кодекске енгiзілгеннен кейiн ғана қолданылуға жатады.', metadata={'source': 'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt_format\\Қазақстан Республикасының Қылмыстық кодексi.txt'}),
 Document(page_content='2. Осы Кодекс Қазақстан Республикасының Конституциясына және халықаралық құқықтың жалпыға бірдей танылған қағидаттары мен нормаларына негiзделедi. Қазақстан Республикасы Конституциясының жоғары заңдық күші бар және Республиканың барлық аумағында тікелей қолданылады. Осы Кодекс пен Қазақстан Республикасы Конституциясының нормалары арасында қайшылықтар болған жағдайда Конституцияның ережелері қолданылады. Осы Кодекстің конституциялық емес деп танылған, оның ішінде адамның және азаматтың Қазақстан Республик

In [107]:
api_key ='sk-proj-4hkSolx4Q6Oas4GMHXdrT3BlbkFJ39PMfFQUBtbrSwrjCKxc'
embedding = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-large')

In [108]:
dbb = FAISS.from_documents(text, embedding)


In [114]:
dbb.similarity_search('адамды өлтірген үшін жаза қандай', k=10)

[Document(page_content='125-бап. Адамды ұрлау\n1. Адамды ұрлау –  \nтөрт жылдан жетi жылға дейiнгi мерзiмге бас бостандығынан айыруға жазаланады.  \n2. Мынадай:  \n1) адамдар тобының алдын ала сөз байласуымен;  \n2) бiрнеше рет;  \n3) өмiрге немесе денсаулыққа қауiптi күш қолданып;  \n4) қаруды немесе қару ретiнде пайдаланылатын заттарды қолданып;  \nЗҚАИ-ның ескертпесі!\n5) тармақты алып тастау көзделген – ҚР 15.04.2024 No 72-VIII (алғашқы ресми жарияланған күнінен кейін күнтізбелік алпыс күн өткен соң қолданысқа енгізіледі) Заңымен.\n5) көрiнеу кәмелетке толмаған адамға қатысты;  \n6) кiнәлi адамға жүктiлiк жағдайда екенi көрінеу белгілі әйелге қатысты;  \n7) екi немесе одан да көп адамға қатысты;  \n8) пайдакүнемдiк ниетпен жасалған;  \n9) адам өзінің қызмет бабын пайдалана отырып жасаған;  \n10) жәбірленушінің жеке басын куәландыратын құжаттарды бұрмалап, жасырып не жойып жасалған дәл сол іс-әрекет –', metadata={'source': 'C:\\Users\\aliha\\Desktop\\nlp\\parser\\txt_form

In [119]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.KAZAKH, Language.RUSSIAN]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
detector.detect_language_of("Менін атым Нур")

Language.KAZAKH

In [121]:
confidence_values = detector.compute_language_confidence_values("мен атым нур")
for language, value in confidence_values:
     print(f"{language.name}: {value:.2f}")

TypeError: cannot unpack non-iterable builtins.ConfidenceValue object

In [141]:
from lingua import Language, LanguageDetectorBuilder
def detect(text):
    languages = [Language.RUSSIAN, Language.KAZAKH]
    detector = LanguageDetectorBuilder.from_languages(*languages).build()
    confidence_values = detector.compute_language_confidence_values(text)
    # confidence_values
    for confidence in confidence_values:
        print(f"{confidence.language.name}: {confidence.value:.2f}")
        if confidence.language.name == "KAZAKH" and confidence.value >= 0.3:
            return 'kz'
    return 'ru'


In [143]:
detect('')

KAZAKH: 1.00


'kz'